In [11]:
import pandas as pd

data = pd.read_csv('./bank.csv', na_values=[' ', '', '-'])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Surname             4897 non-null   object 
 1   CreditScore         4898 non-null   int64  
 2   Geography           4896 non-null   object 
 3   Gender              4898 non-null   object 
 4   Age                 4895 non-null   float64
 5   Tenure              4898 non-null   int64  
 6   Balance             4898 non-null   float64
 7   NumOfProducts       4898 non-null   int64  
 8   HasCrCard           4898 non-null   int64  
 9   IsActiveMember      4898 non-null   int64  
 10  EstimatedSalary     4898 non-null   float64
 11  Exited              4898 non-null   int64  
 12  Satisfaction.Score  4898 non-null   int64  
 13  Card.Type           4898 non-null   object 
 14  Point.Earned        4898 non-null   int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 574.1+

In [13]:
data['Stayed'] = ~data['Exited']

In [ ]:
data = data[data['Age'] <= 87].reset_index(drop=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4894 entries, 1 to 4897
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Surname             4893 non-null   object 
 1   CreditScore         4894 non-null   int64  
 2   Geography           4892 non-null   object 
 3   Gender              4894 non-null   object 
 4   Age                 4894 non-null   float64
 5   Tenure              4894 non-null   int64  
 6   Balance             4894 non-null   float64
 7   NumOfProducts       4894 non-null   int64  
 8   HasCrCard           4894 non-null   int64  
 9   IsActiveMember      4894 non-null   int64  
 10  EstimatedSalary     4894 non-null   float64
 11  Exited              4894 non-null   int64  
 12  Satisfaction.Score  4894 non-null   int64  
 13  Card.Type           4894 non-null   object 
 14  Point.Earned        4894 non-null   int64  
 15  Stayed              4894 non-null   int64  
dtypes: float64(